This notebook is part of my [Python data science curriculum](http://www.terran.us/articles/python_curriculum.html)

In [1]:
import numpy as np
import pandas as pd

This document uses the ["collapsible headings" extension from nbextensions](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/collapsible_headings/readme.html).  If you have that extension installed, you will see that the "solution" sections start collapsed, and you can expand them and look at them _after_ doing the exercise yourself.  I also like the "toc2" extension, which will number each section automatically and create a table of contents; it's helpful in a document this long.

I tried the "exercise2" extension, but it had unacceptable bugs (the solution became irretrievable if I inserted a new cell right below the collapsed exercise) so I cannot recommend it.

# Indexing

The heuristic indexing in Numpy (that's [] for Series and ix for DataFrame) is a disaster.  In fact, Pandas has deprecated ix because it was so bad, and you should never use [] on Series for the same reason.  Always use iloc and loc, which is the only way to be retain your sanity.

Other languages have heuristic indexing which works fine and is entirely comprehensible, so I was initially skeptical about the decision to deprecate this functionality in Pandas.  I investigated it extensively and I can assure you that it is mindbogglingly bad, to the point that one wonders how any sane person could possibly have made these design choices.  Refer to the separate document "Indexing" to see just how terrible these behaviors are: they depend not only on the type of the index, but also on whether or not it contains duplicate keys and whether the keys are in monotonic order.

In this class, we will use .loc and .iloc indexing as much as possible.  The main thing you need to understand to read existing code is that a single index on a dataframe containing a single string selects a column, whereas any slice selects rows.

## Exercise

Create a data frame with at least 3 rows and columns.  Give it a named row index.  Make all columns numeric, with at least one integer and one floating point column.

## Solution

In [2]:
d = pd.DataFrame(data={'w':np.arange(0,5), 'x':np.arange(0,55,11), 'y':np.random.normal(0,1,5)}, index=['a','b','c','d','e'])
d

,w,x,y
a,0,0,2.695857
b,1,11,0.676350
c,2,22,0.002345
d,3,33,-0.665981
e,4,44,1.919519


## Exercise

Select a single column by name.  Do so in a way that returns that column as a Series, then do it again in a way that returns it as a one-column dataframe.

## Solution

In [3]:
# As a series:
d.loc[:,'w']

a    0
b    1
c    2
d    3
e    4
Name: w, dtype: int64

In [4]:
# As a dataframe
d.loc[:,'w':'w']
d.loc[:,['w']]

,w
a,0
b,1
c,2
d,3
e,4


In [5]:
# Here are some other solutions which use single bracket indexing.  Although I don't generally
# like single-bracket indexing because it has too many heuristics, selecting a single column is
# very commonly used and seems OK.
d['w']
d[['w']]

# this DOESN'T work - slices try to select by row:
d['w':'w']


,w,x,y


## Exercise

Select a single row by name.  Do so in a way that returns the row as a Series, then do it again in a way that returns a one-row dataframe.

## Solution

In [6]:
# Series.  Note that type casting can occur here, because your columns may not all be the same type, but a Series must be.
d.loc['b',:]

w     1.00000
x    11.00000
y     0.67635
Name: b, dtype: float64

In [7]:
# 1-row DF - no type-casting occurs:
d.loc[['b'],:]

,w,x,y
b,1,11,0.67635


In [8]:
# and here are the bad answers.
# This will give us a one-row DF:
d['b':'b']
# but how can we get it as a series?  Can we?
# This won't work - it tries to look up a column instead:
#d['b']

,w,x,y
b,1,11,0.67635


## Exercise

Select all rows where one of your integer columns is odd.

## Solution

In [9]:
d.loc[d['x']%2==1]

,w,x,y
b,1,11,0.676350
d,3,33,-0.665981


## Exercise

Select a single cell using implicit indexing (0-based integers).  Do it 4 ways, to get each of these result types:

- As a scalar
- As a 1-element Series from the row
- As a 1-element Series from the column
- As a 1x1 data frame

## Solution

In [10]:
# Scalar
d.iloc[1,1]

11

In [11]:
# Series from row
d.iloc[1:2,1]

b    11
Name: x, dtype: int64

In [12]:
# Series from column - note different metadata name and element name, and also note that in this case, type casting has
# occurred even though we are not returning data from the column which required it:
d.iloc[1,1:2]

x    11.0
Name: b, dtype: float64

In [13]:
# 1x1 DF:
d.iloc[1:2,1:2]

,x
b,11


In [14]:
# 1-element lists are equivalent to 1-element slices
d.iloc[[1], [1]]

,x
b,11


## Exercise

The only tricky part is when you want to index rows by name and columns by number or vice versa.
Some approaches are shown here:
http://pandas-docs.github.io/pandas-docs-travis/indexing.html#ix-indexer-is-deprecated
but I personally find that those approaches expose too much implementation detail and require multiple references to the object, so I don't like them.  Using either those approach or something else, implement the selection of a single cell where the column is chosen by name and the row is chosen by number.  As before, produce all 4 types of result.

## Solution

In [15]:
# I prefer to simply index in two steps.  This will get us the 1x1 df:
d.iloc[[1],:].loc[:,['w']]

,w
b,1


In [16]:
# If we drop a dimension in the first index, note that the 2nd is then a 1d series.  This gives us a scalar:
d.iloc[1,:].loc['w']

1.0

In [17]:
# This gives us a column-derived Series
d.iloc[[1],:].loc[:,'w']

b    1
Name: w, dtype: int64

In [18]:
# This gives us the row-derived series, while keeping the operations in the same order:
d.iloc[1,:].loc[['w']]

w    1.0
Name: b, dtype: float64

In [19]:
# The difference in time between these approaches is probably not important for most purposes
%timeit -n100 d.iloc[[1],:].loc[:,['w']]
%timeit -n100 d.iloc[[1], d.columns.get_indexer(['w'])]
%timeit -n100 d.loc[[d.index[1]], ['w']]
# This will produce a deprecation warning
%timeit -n100 d.ix[[1],['w']]

1.11 ms ± 57.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
933 µs ± 67.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.12 ms ± 73.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


/Users/terran/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


813 µs ± 55.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Index Alignment

## Exercise

Create another data frame which shares some but not all of the row and column lables of your previous one.  Add the two together.

## Solution

In [20]:
e = pd.DataFrame(data={'l':np.arange(0,5), 'x':np.arange(0,55,11), 'y':np.random.normal(0,1,5)}, index=['a','b','z','q','e'])
e

,l,x,y
a,0,0,-0.263012
b,1,11,-0.719347
z,2,22,0.780059
q,3,33,-0.011191
e,4,44,1.141120


In [21]:
# Note also that the integers have been promoted to floats, because there is no NA for integers in Pandas.
d+e

,l,w,x,y
a,NaN,NaN,0.0,2.432846
b,NaN,NaN,22.0,-0.042997
c,NaN,NaN,NaN,NaN
d,NaN,NaN,NaN,NaN
e,NaN,NaN,88.0,3.060639
q,NaN,NaN,NaN,NaN
z,NaN,NaN,NaN,NaN


## Exercise

Numpy and Pandas made different choices about how NaNs are handled, which we will explore here.  First, create a Numpy array which contains some numbers and some NaN values.  Sum it with sum.  Do you get a value?  Now, convert your array to a Pandas Series, and sum it with sum().

In your opinion, which library made the correct decision, and why?

## Solution


In [22]:
# Start with an array:
a = np.array([1,2,3,np.nan])
a.sum()

nan

In [23]:
# If we want this sum to work, we have to use the special nansum function:
np.nansum(a)

6.0

In [24]:
# but if we convert it to a series, NaNs are ignored in the sum
pd.Series(a).sum()

6.0

In [25]:
# Note that it's the data type that makes the difference - we can explicitly invoke numpy.sum on
# a pandas Series and the result is not NaN:
np.sum(pd.Series(a))

6.0

## Exercise

Return to your data frame from two exercises ago, the sum of dataframes with partial overlap in row and column names.  Scale and center the columns of this data frame (this means subtract the mean and divide by the sd).  You will have to handle NA values.

## Solution

In [26]:
f=d+e
# Note that std is a numpy function, which has to be explicitly applied.  There is no .std member on a dataframe.
(f-f.mean())/f.apply(np.mean)

,l,w,x,y
a,NaN,NaN,-1.0,0.339061
b,NaN,NaN,-0.4,-1.023666
c,NaN,NaN,NaN,NaN
d,NaN,NaN,NaN,NaN
e,NaN,NaN,1.4,0.684605
q,NaN,NaN,NaN,NaN
z,NaN,NaN,NaN,NaN


It's important to note here that Numpy does not follow the usual convention for standard deviation.  Most tools divide by N-1, to get the unbiased estimate of the population standard deviation from the sample; Numpy defaults to N-0, which is correct if the data represents the full population.  This is configurable with the ddof argument and will make more sense after we cover statistics; for now, just understand that the standard deviation you get from Numpy will not match the answer from most other tools when you have a small number of values.

In [27]:
# If for some reason you wanted row scaling, you'd have to do it with lots of explicit axis arguments:
f.subtract(f.mean(axis=1),axis=0).divide(f.apply(np.std,axis=1),axis=0)

,l,w,x,y
a,NaN,NaN,-1.0,1.0
b,NaN,NaN,1.0,-1.0
c,NaN,NaN,NaN,NaN
d,NaN,NaN,NaN,NaN
e,NaN,NaN,1.0,-1.0
q,NaN,NaN,NaN,NaN
z,NaN,NaN,NaN,NaN


## Exercise

Drop all rows which contain only NaN values.  Then drop all columns which contain only NaN values.

## Solution

In [28]:
f.dropna(axis=0,how='all').dropna(axis=1,how='all')

,x,y
a,0.0,2.432846
b,22.0,-0.042997
e,88.0,3.060639


In [29]:
# You used to be able to do this, but it's now deprecated because it had bugs when used along with other arguments
# https://github.com/pandas-dev/pandas/issues/20987
# f.dropna(axis=[0,1],how='all')

In [30]:
from plotnine.data import diamonds

# Concatenation

## Exercise

Return to your two data frames which have some overlapping and some non-overlapping columns.  Concatenate them vertically, allowing any missing columns to be filled with NAs.  Then do the same thing concatenating them horizontally.

## Solution

In [31]:
# append always uses axis 0 (rows) and does not take an axis parameter:
d.append(e)

,l,w,x,y
a,NaN,0.0,0,2.695857
b,NaN,1.0,11,0.676350
c,NaN,2.0,22,0.002345
d,NaN,3.0,33,-0.665981
e,NaN,4.0,44,1.919519
a,0.0,NaN,0,-0.263012
b,1.0,NaN,11,-0.719347
z,2.0,NaN,22,0.780059
q,3.0,NaN,33,-0.011191
e,4.0,NaN,44,1.141120


In [32]:
# Sort must be explicitly set to T or F to avoid a warning
pd.concat([d,e],axis=1,sort=False)

,w,x,y,l,x,y
a,0.0,0.0,2.695857,0.0,0.0,-0.263012
b,1.0,11.0,0.676350,1.0,11.0,-0.719347
c,2.0,22.0,0.002345,NaN,NaN,NaN
d,3.0,33.0,-0.665981,NaN,NaN,NaN
e,4.0,44.0,1.919519,4.0,44.0,1.141120
z,NaN,NaN,NaN,2.0,22.0,0.780059
q,NaN,NaN,NaN,3.0,33.0,-0.011191


## Exercise

Now join them keeping only the columns which are in common, and then keeping all the columns from the first df.

## Solution

In [33]:
pd.concat([d,e],join='inner')

,x,y
a,0,2.695857
b,11,0.676350
c,22,0.002345
d,33,-0.665981
e,44,1.919519
a,0,-0.263012
b,11,-0.719347
z,22,0.780059
q,33,-0.011191
e,44,1.141120


In [34]:
# For keeping the columns from the first, you'd like to do this, but you can't:
try:
    pd.concat([d,e], join='left')
except Exception as ex:
    print(repr(ex))
    
# You have to explicitly specify which columns to keep instead
pd.concat([d,e], join_axes=[d.columns])

ValueError('Only can inner (intersect) or outer (union) join the other axis',)


,w,x,y
a,0.0,0,2.695857
b,1.0,11,0.676350
c,2.0,22,0.002345
d,3.0,33,-0.665981
e,4.0,44,1.919519
a,NaN,0,-0.263012
b,NaN,11,-0.719347
z,NaN,22,0.780059
q,NaN,33,-0.011191
e,NaN,44,1.141120


## Exercise

Now do an inner join on the columns, and add a level of indexing to disambiguate the rows with duplicate names.

## Solution

In [35]:
pd.concat([d,e],keys=['d','e'],join='inner',sort=False)

x         y
d a   0  2.695857
  b  11  0.676350
  c  22  0.002345
  d  33 -0.665981
  e  44  1.919519
e a   0 -0.263012
  b  11 -0.719347
  z  22  0.780059
  q  33 -0.011191
  e  44  1.141120

# Diamonds Data

Install the plotnine package (a port of ggplot) and import the diamonds dataset.  In October 2018, I found that some of plotnine's dependencies won't build on Python 3.7, and I had to go back to 3.6 to get it to install.

In [36]:
from plotnine.data import diamonds

# Multi-Indexes

## Exercise

Create an index on cut, color, and clarity.  Sort the data by this index.

## Solution

In [37]:
d2 = diamonds.set_index(['cut','color','clarity']).sort_index()

## Exercise

Using your new index, select all diamonds of cut "Fair" and colors "D" and "F".

## Solution

In [38]:
d2.loc[(['Fair'],['D','F'],),:]

carat  depth  table  price     x     y     z
cut  color clarity                                              
Fair D     I1        1.50   64.7   62.0   5460  7.19  7.04  4.60
           I1        1.70   64.7   56.0   5617  7.46  7.37  4.80
           I1        3.40   66.8   52.0  15964  9.42  9.34  6.27
           I1        0.91   66.2   57.0   2491  6.00  5.94  3.95
           SI2       0.75   64.6   57.0   2848  5.74  5.72  3.70
           SI2       0.90   66.9   57.0   2885  6.02  5.90  3.99
           SI2       1.00   69.3   58.0   2974  5.96  5.87  4.10
           SI2       1.01   64.6   56.0   3003  6.31  6.24  4.05
           SI2       0.91   62.5   66.0   3079  6.08  6.01  3.78
           SI2       0.90   65.9   59.0   3205  6.00  5.95  3.94
           SI2       0.90   66.0   58.0   3205  6.00  5.97  3.95
           SI2       0.90   64.7   54.0   3205  6.10  6.04  3.93
           SI2       0.90   65.7   60.0   3205  5.98  5.93  3.91
           SI2       0.90   64.7   59.0   3205  6.09  5.99  3.91
           SI2       1.00   64.8   60.0   3304  6.23  6.18  4.02
           SI2       0.95   64.4   60.0   3384  6.06  6.02  3.89
           SI2       0.90   64.9   57.0   3473  6.03  5.98  3.90
           SI2       0.90   64.5   61.0   3473  6.10  6.00  3.90
           SI2       1.00   65.2   56.0   3634  6.27  6.21  4.07
           SI2       1.03   66.4   56.0   3743  6.31  6.19  4.15
           SI2       1.00   65.6   66.0   3767  6.10  6.01  3.97
           SI2       1.04   65.8   58.0   3780  6.32  6.25  4.13
           SI2       1.05   65.4   59.0   3816  6.30  6.24  4.10
           SI2       0.90   64.6   59.0   3847  6.04  6.01  3.89
           SI2       1.01   64.7   57.0   3871  6.31  6.27  4.07
           SI2       1.18   57.0   66.0   3899  6.96  6.91  3.98
           SI2       1.00   65.0   56.0   3965  6.27  6.23  4.06
           SI2       1.00   66.5   59.0   3965  6.24  6.21  4.14
           SI2       1.01   65.3   55.0   4205  6.33  6.19  4.09
           SI2       1.00   64.5   54.0   4312  6.34  6.28  4.07
...                   ...    ...    ...    ...   ...   ...   ...
     F     VS1       0.50   62.9   68.0   1348  5.02  4.96  3.14
           VS1       0.50   56.4   62.0   1446  5.31  5.22  2.97
           VS1       0.50   65.0   55.0   1577  5.01  4.96  3.24
           VS1       0.52   57.2   59.0   1784  5.35  5.39  3.07
           VS1       0.57   64.5   61.0   1788  5.23  5.19  3.36
           VS1       0.62   60.0   64.0   2083  5.69  5.48  3.36
           VS1       0.70   64.5   57.0   2381  5.65  5.58  3.62
           VS1       0.70   67.8   60.0   2381  5.50  5.39  3.69
           VS1       0.70   65.8   59.0   2381  5.58  5.48  3.64
           VS1       0.70   55.9   62.0   2399  5.94  5.83  3.29
           VS1       0.71   57.3   67.0   2707  5.96  5.76  3.36
           VVS2      0.51   65.4   60.0   3920  4.98  4.90  3.23
           VVS2      0.51   60.7   66.0   4368  5.21  5.11  3.13
           VVS2      1.02   64.8   58.0   6863  6.22  6.28  4.05
           VVS2      1.50   64.7   53.0  16364  7.22  7.15  4.65
           VVS2      0.33   65.7   54.0    693  4.37  4.30  2.85
           VVS2      0.30   65.5   61.0    743  4.19  4.15  2.73
           VVS2      0.31   56.9   59.0    757  4.45  4.48  2.54
           VVS2      0.57   65.2   56.2   1902  5.19  5.23  3.40
           VVS2      0.62   57.1   60.0   2088  5.65  5.70  3.24
           VVS2      0.60   60.1   66.0   2484  5.48  5.44  3.28
           VVS1      1.02   64.6   59.0   7960  6.31  6.26  4.06
           VVS1      1.32   58.0   66.0  12648  7.31  7.28  4.23
           VVS1      0.31   63.1   61.0    740  4.32  4.37  2.74
           VVS1      0.35   54.6   59.0   1011  4.85  4.79  2.63
           VVS1      0.34   55.2   59.0   1040  4.72  4.77  2.62
           IF        0.62   60.1   61.0   2861  5.53  5.56  3.33
           IF        0.71   58.7   62.0   3205  5.87  5.81  3.43
           IF        0.37   52.3   61.0   1166  4.96  4

## Exercise

Using your multi-index, select all diamonds of all cuts, and colors D and E.

## Solution

In [39]:
idx = pd.IndexSlice

d2.loc[idx[:,'D':'E',:], :]

carat  depth  table  price     x     y     z
cut   color clarity                                              
Fair  D     I1        1.50   64.7   62.0   5460  7.19  7.04  4.60
            I1        1.70   64.7   56.0   5617  7.46  7.37  4.80
            I1        3.40   66.8   52.0  15964  9.42  9.34  6.27
            I1        0.91   66.2   57.0   2491  6.00  5.94  3.95
            SI2       0.75   64.6   57.0   2848  5.74  5.72  3.70
            SI2       0.90   66.9   57.0   2885  6.02  5.90  3.99
            SI2       1.00   69.3   58.0   2974  5.96  5.87  4.10
            SI2       1.01   64.6   56.0   3003  6.31  6.24  4.05
            SI2       0.91   62.5   66.0   3079  6.08  6.01  3.78
            SI2       0.90   65.9   59.0   3205  6.00  5.95  3.94
            SI2       0.90   66.0   58.0   3205  6.00  5.97  3.95
            SI2       0.90   64.7   54.0   3205  6.10  6.04  3.93
            SI2       0.90   65.7   60.0   3205  5.98  5.93  3.91
            SI2       0.90   64.7   59.0   3205  6.09  5.99  3.91
            SI2       1.00   64.8   60.0   3304  6.23  6.18  4.02
            SI2       0.95   64.4   60.0   3384  6.06  6.02  3.89
            SI2       0.90   64.9   57.0   3473  6.03  5.98  3.90
            SI2       0.90   64.5   61.0   3473  6.10  6.00  3.90
            SI2       1.00   65.2   56.0   3634  6.27  6.21  4.07
            SI2       1.03   66.4   56.0   3743  6.31  6.19  4.15
            SI2       1.00   65.6   66.0   3767  6.10  6.01  3.97
            SI2       1.04   65.8   58.0   3780  6.32  6.25  4.13
            SI2       1.05   65.4   59.0   3816  6.30  6.24  4.10
            SI2       0.90   64.6   59.0   3847  6.04  6.01  3.89
            SI2       1.01   64.7   57.0   3871  6.31  6.27  4.07
            SI2       1.18   57.0   66.0   3899  6.96  6.91  3.98
            SI2       1.00   65.0   56.0   3965  6.27  6.23  4.06
            SI2       1.00   66.5   59.0   3965  6.24  6.21  4.14
            SI2       1.01   65.3   55.0   4205  6.33  6.19  4.09
            SI2       1.00   64.5   54.0   4312  6.34  6.28  4.07
...                    ...    ...    ...    ...   ...   ...   ...
Ideal E     IF        0.32   61.9   56.0   1170  4.42  4.40  2.73
            IF        0.31   60.4   56.0   1188  4.43  4.38  2.66
            IF        0.35   61.4   56.0   1197  4.53  4.56  2.79
            IF        0.35   62.2   56.0   1197  4.51  4.52  2.81
            IF        0.33   61.9   57.0   1207  4.45  4.41  2.74
            IF        0.31   61.5   56.0   1214  4.35  4.40  2.69
            IF        0.31   62.5   56.0   1214  4.34  4.37  2.72
            IF        0.31   63.0   55.0   1214  4.34  4.37  2.73
            IF        0.31   61.8   54.0   1214  4.36  4.38  2.70
            IF        0.31   60.4   56.0   1239  4.43  4.38  2.66
            IF        0.34   62.0   56.0   1243  4.51  4.49  2.79
            IF        0.34   61.7   56.0   1243  4.48  4.46  2.76
            IF        0.34   62.0   56.0   1243  4.50  4.47  2.78
            IF        0.39   62.5   57.0   1248  4.70  4.74  2.95
            IF        0.35   61.4   56.0   1254  4.56  4.53  2.79
            IF        0.35   62.2   56.0   1254  4.52  4.51  2.81
            IF        0.33   62.1   55.0   1289  4.43  4.46  2.76
            IF        0.32   61.2   57.0   1300  4.40  4.43  2.70
            IF        0.38   61.4   55.0   1300  4.68  4.70  2.88
            IF        0.31   61.0   57.0   1319  4.36  4.39  2.67
            IF        0.39   61.3   56.0   1333  4.71  4.75  2.90
            IF        0.39   61.1   57.0   1333  4.73  4.76  2.90
            IF        0.32   61.4   56.0   1359  4.38  4.42  2.70
            IF        0.38   61.4   55.0   1402  4.68  4.74  2.89
            IF        0.38   62.2   54.1   1402  4.64  4.67  2.89
            IF        0.39   61.5   55.5   1418  4.68  4.72  2.88
            IF        0.41   62.1   54.0   1419  4.75  4.81  2.97
            IF        0.38   62.7   55.0   1433  4.61  4.67  

# Aggregation

## Exercise

Take the diamonds dataframe.  Calculate the mean of carat and price for each value of clarity.

## Solution

In [40]:
diamonds.groupby('clarity').mean().loc[:,['carat','price']]

,carat,price
clarity,,
I1,1.283846,3924.168691
SI2,1.077648,5063.028606
SI1,0.850482,3996.001148
VS2,0.763935,3924.989395
VS1,0.727158,3839.455391
VVS2,0.596202,3283.737071
VVS1,0.503321,2523.114637
IF,0.505123,2864.839106


## Exercise

Now calculate the min, mean, median, and max for each of carat and price, grouped as before.

## Solution

In [41]:
# We'd like to do this, but we can't:
try:
    diamonds.groupby('clarity').loc[:,['carat','price']].aggregate([min,np.mean,np.median,max])
except Exception as ex:
    print(repr(ex))

# Sadly, we are forced to use the bracket syntax instead of loc/iloc, which means dealing with all of its heuristics.
# A list of columns will work, but a slice wouldn't:
diamonds.groupby('clarity')[['carat','price']].aggregate([min,np.mean,np.median,max])

# Note that you can also put a lambda or a quoted function name in the aggregate() call.

AttributeError("Cannot access callable attribute 'loc' of 'DataFrameGroupBy' objects, try using the 'apply' method",)


carat                        price                           
          min      mean median   max   min         mean median    max
clarity                                                              
I1       0.30  1.283846   1.12  5.01   345  3924.168691   3344  18531
SI2      0.20  1.077648   1.01  3.04   326  5063.028606   4072  18804
SI1      0.21  0.850482   0.76  2.57   326  3996.001148   2822  18818
VS2      0.20  0.763935   0.63  3.51   334  3924.989395   2054  18823
VS1      0.23  0.727158   0.57  2.59   327  3839.455391   2005  18795
VVS2     0.23  0.596202   0.44  2.07   336  3283.737071   1311  18768
VVS1     0.23  0.503321   0.39  2.31   336  2523.114637   1093  18777
IF       0.23  0.505123   0.35  2.29   369  2864.839106   1080  18806

## Exercise

For all combinations of cut, color, and clarity, count the number of rows in the diamonds dataset using groupby.  Present it in a readable way.

Then do the same thing using pivot_table.  Make your result exactly match.

## Solution

In [42]:
diamonds.groupby(['cut','color','clarity']).size().unstack().fillna(0)

# You could also do it this way, as a demo of apply:
# diamonds.groupby(['cut','color','clarity']).apply(lambda x: x.shape[0]).unstack().fillna(0)

# Or, taking advantage of the fact that we already have a dataset with a multiindex on these columns:
# d2.groupby(level=[0,1,2]).size().unstack().fillna(0)


clarity            I1    SI2    SI1     VS2    VS1   VVS2   VVS1     IF
cut       color                                                        
Fair      D       4.0   56.0   58.0    25.0    5.0    9.0    3.0    3.0
          E       9.0   78.0   65.0    42.0   14.0   13.0    3.0    0.0
          F      35.0   89.0   83.0    53.0   33.0   10.0    5.0    4.0
          G      53.0   80.0   69.0    45.0   45.0   17.0    3.0    2.0
          H      52.0   91.0   75.0    41.0   32.0   11.0    1.0    0.0
          I      34.0   45.0   30.0    32.0   25.0    8.0    1.0    0.0
          J      23.0   27.0   28.0    23.0   16.0    1.0    1.0    0.0
Good      D       8.0  223.0  237.0   104.0   43.0   25.0   13.0    9.0
          E      23.0  202.0  355.0   160.0   89.0   52.0   43.0    9.0
          F      19.0  201.0  273.0   184.0  132.0   50.0   35.0   15.0
          G      19.0  163.0  207.0   192.0  152.0   75.0   41.0   22.0
          H      14.0  158.0  235.0   138.0   77.0   45.0   31.0    4.0
          I       9.0   81.0  165.0   110.0  103.0   26.0   22.0    6.0
          J       4.0   53.0   88.0    90.0   52.0   13.0    1.0    6.0
Very Good D       5.0  314.0  494.0   309.0  175.0  141.0   52.0   23.0
          E      22.0  445.0  626.0   503.0  293.0  298.0  170.0   43.0
          F      13.0  343.0  559.0   466.0  293.0  249.0  174.0   67.0
          G      16.0  327.0  474.0   479.0  432.0  302.0  190.0   79.0
          H      12.0  343.0  547.0   376.0  257.0  145.0  115.0   29.0
          I       8.0  200.0  358.0   274.0  205.0   71.0   69.0   19.0
          J       8.0  128.0  182.0   184.0  120.0   29.0   19.0    8.0
Premium   D      12.0  421.0  556.0   339.0  131.0   94.0   40.0   10.0
          E      30.0  519.0  614.0   629.0  292.0  121.0  105.0   27.0
          F      34.0  523.0  608.0   619.0  290.0  146.0   80.0   31.0
          G      46.0  492.0  566.0   721.0  566.0  275.0  171.0   87.0
          H      46.0  521.0  655.0   532.0  336.0  118.0  112.0   40.0
          I      24.0  312.0  367.0   315.0  221.0   82.0   84.0   23.0
          J      13.0  161.0  209.0   202.0  153.0   34.0   24.0   12.0
Ideal     D      13.0  356.0  738.0   920.0  351.0  284.0  144.0   28.0
          E      18.0  469.0  766.0  1136.0  593.0  507.0  335.0   79.0
          F      42.0  453.0  608.0   879.0  616.0  520.0  440.0  268.0
          G      16.0  486.0  660.0   910.0  953.0  774.0  594.0  491.0
          H      38.0  450.0  763.0   556.0  467.0  289.0  326.0  226.0
          I      17.0  274.0  504.0   438.0  408.0  178.0  179.0   95.0
          J       2.0  110.0  243.0   232.0  201.0   54.0   29.0   25.0

In [43]:
# with pivot_table.  Note that the selection of 'carat' for the value column is meaningless, since
# only the count of rows is used, but the selection of _some_ column appears to be required.  Leaving it
# off selects _all_ columns, producing redundant results.
d2.pivot_table(values='carat', index=['cut','color'], columns=['clarity'], aggfunc=np.size).fillna(0)

clarity            I1    SI2    SI1     VS2    VS1   VVS2   VVS1     IF
cut       color                                                        
Fair      D       4.0   56.0   58.0    25.0    5.0    9.0    3.0    3.0
          E       9.0   78.0   65.0    42.0   14.0   13.0    3.0    0.0
          F      35.0   89.0   83.0    53.0   33.0   10.0    5.0    4.0
          G      53.0   80.0   69.0    45.0   45.0   17.0    3.0    2.0
          H      52.0   91.0   75.0    41.0   32.0   11.0    1.0    0.0
          I      34.0   45.0   30.0    32.0   25.0    8.0    1.0    0.0
          J      23.0   27.0   28.0    23.0   16.0    1.0    1.0    0.0
Good      D       8.0  223.0  237.0   104.0   43.0   25.0   13.0    9.0
          E      23.0  202.0  355.0   160.0   89.0   52.0   43.0    9.0
          F      19.0  201.0  273.0   184.0  132.0   50.0   35.0   15.0
          G      19.0  163.0  207.0   192.0  152.0   75.0   41.0   22.0
          H      14.0  158.0  235.0   138.0   77.0   45.0   31.0    4.0
          I       9.0   81.0  165.0   110.0  103.0   26.0   22.0    6.0
          J       4.0   53.0   88.0    90.0   52.0   13.0    1.0    6.0
Very Good D       5.0  314.0  494.0   309.0  175.0  141.0   52.0   23.0
          E      22.0  445.0  626.0   503.0  293.0  298.0  170.0   43.0
          F      13.0  343.0  559.0   466.0  293.0  249.0  174.0   67.0
          G      16.0  327.0  474.0   479.0  432.0  302.0  190.0   79.0
          H      12.0  343.0  547.0   376.0  257.0  145.0  115.0   29.0
          I       8.0  200.0  358.0   274.0  205.0   71.0   69.0   19.0
          J       8.0  128.0  182.0   184.0  120.0   29.0   19.0    8.0
Premium   D      12.0  421.0  556.0   339.0  131.0   94.0   40.0   10.0
          E      30.0  519.0  614.0   629.0  292.0  121.0  105.0   27.0
          F      34.0  523.0  608.0   619.0  290.0  146.0   80.0   31.0
          G      46.0  492.0  566.0   721.0  566.0  275.0  171.0   87.0
          H      46.0  521.0  655.0   532.0  336.0  118.0  112.0   40.0
          I      24.0  312.0  367.0   315.0  221.0   82.0   84.0   23.0
          J      13.0  161.0  209.0   202.0  153.0   34.0   24.0   12.0
Ideal     D      13.0  356.0  738.0   920.0  351.0  284.0  144.0   28.0
          E      18.0  469.0  766.0  1136.0  593.0  507.0  335.0   79.0
          F      42.0  453.0  608.0   879.0  616.0  520.0  440.0  268.0
          G      16.0  486.0  660.0   910.0  953.0  774.0  594.0  491.0
          H      38.0  450.0  763.0   556.0  467.0  289.0  326.0  226.0
          I      17.0  274.0  504.0   438.0  408.0  178.0  179.0   95.0
          J       2.0  110.0  243.0   232.0  201.0   54.0   29.0   25.0

# Joins

The join operations provided are identical to those in SQL, so they should be quite straightforward conceptually; there is just a bit of syntax to memorize.

## Exercise

Suppose that you did not have a reference for the "clarity" levels in the diamonds dataset, which are not named in a way that is obvious to those without domain knowledge.  By looking only at diamonds around the same size, use the price to infer the ordering of clarity from worst to best and create a table of rankings.  Join this back to the original dataset. 

## Solution

In [44]:
tmp=diamonds.query('carat > 0.95 & carat < 1.05').groupby('clarity').aggregate({'price':np.mean}).sort_values('price')
tmp

,price
clarity,
I1,2695.344595
SI2,4131.280856
SI1,4866.168311
VS2,5946.731788
VS1,6597.966049
VVS2,8270.857534
VVS1,8959.781955
IF,10981.702381


In [45]:
tmp=pd.merge(tmp,tmp.rank(),on='clarity')
tmp.columns=['price','clarity_rank']
tmp

,price,clarity_rank
clarity,,
I1,2695.344595,1.0
SI2,4131.280856,2.0
SI1,4866.168311,3.0
VS2,5946.731788,4.0
VS1,6597.966049,5.0
VVS2,8270.857534,6.0
VVS1,8959.781955,7.0
IF,10981.702381,8.0


In [46]:
pd.merge(diamonds,tmp.drop('price',axis='columns'),on='clarity').head()

,carat,cut,color,clarity,depth,table,price,x,y,z,clarity_rank
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,2.0
1,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,2.0
2,0.31,Ideal,J,SI2,62.2,54.0,344,4.35,4.37,2.71,2.0
3,0.20,Premium,E,SI2,60.2,62.0,345,3.79,3.75,2.27,2.0
4,0.30,Ideal,I,SI2,62.0,54.0,348,4.31,4.34,2.68,2.0


# Time Series

## Exercise

Create a time series representing returns from a theoretical investment.  This investment has an average return of 7%/year, plus a lot of noise.  You can use a normal distribution for the noise since that's the only one you've seen so far, althoubh be aware that most real investments have fatter tails than that.  Also, make sure there are returns only for business days (stock market days would be even better, but that can't be easily generated without an external calendar).  Generate 10 years of returns.

## Solution

In [47]:
# Start with the dates, indexing by business days
i=pd.date_range(start='2008-01-01', end='2017-12-31', freq='B')
# Get the number of days in the year, so we can calculate a daily amount which comes
# to the required 3% annually.
days_per_year=len(i)/10
r=np.random.normal(1.07**(1/days_per_year),.005,len(i))
ret = pd.DataFrame({'date':i, 'return':r})
del i
del r
ret.set_index('date',inplace=True)
ret.head()

,return
date,
2008-01-01,1.010281
2008-01-02,1.013070
2008-01-03,1.001346
2008-01-04,0.997003
2008-01-07,0.992043


## Exercise

Now, using your daily returns, calculate a cumulative value of $1 invested at the beginning of the time series.

## Solution

In [48]:
# We might try this, but the prod or product function doesn't exist!
#ret['value']=ret['return'].expanding(1).prod()

# but cumprod on a series exists
ret['value']=ret['return'].cumprod()
ret.head()

,return,value
date,,
2008-01-01,1.010281,1.010281
2008-01-02,1.013070,1.023486
2008-01-03,1.001346,1.024864
2008-01-04,0.997003,1.021792
2008-01-07,0.992043,1.013662


## Exercise

It's more common to have to go the other direction - take the value and calculate the daily return.  Recalculate the return using your daily value.

## Solution

In [51]:
# Start with a lagged value series
ret['value1']=ret['value'].shift(1)
# Fill the initial NA with a 1
ret['value1'].fillna(1,inplace=True)
# and divide!
ret['return_bis'] = ret['value']/ret['value1']

# Make sure it matches
np.allclose(ret['return'], ret['return_bis'])

True

In [87]:
# Alternative approach.  Note that pct_change is NOT A PERCENT; it's a proportion,
# it just has the wrong name.
ret['return_bis']=ret['value'].pct_change()+1
# Fill in the first NA
ret['return_bis'].fillna(ret['value'],inplace=True)
# Note that "return" is a reserved word, so we can't say ret.return
np.allclose(ret['return'],ret.return_bis)

True

## Exercise

At each of the daily, weekly, monthly, and yearly level, calculate what proportion of your returns are positive.

## Solution

In [50]:
pd.DataFrame({'positive':{
    'day': np.mean(ret['return']>1),
    'week': np.mean(ret['return'].resample('W').prod() > 1),
    'month': np.mean(ret['return'].resample('M').prod() > 1),
    'qtr': np.mean(ret['return'].resample('Q').prod() > 1),
    'year': np.mean(ret['return'].resample('Y').prod() > 1)

}})


,positive
day,0.517440
month,0.600000
qtr,0.700000
week,0.519157
year,0.800000


In [74]:
pd.Series([1,1,1,2,1]).pct_change()

0    NaN
1    0.0
2    0.0
3    1.0
4   -0.5
dtype: float64